
# Run your first Qiskit Serverless workload remotely

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/serverless-run-first-workload)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/serverless-run-first-workload)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=1.3.2
qiskit-ibm-runtime~=0.34.0
qiskit-aer~=0.15.1
qiskit-serverless~=0.18.1
qiskit-ibm-catalog~=0.2
qiskit-addon-sqd~=0.9.0
qiskit-addon-utils~=0.1.0
qiskit-addon-mpf~=0.2.0
qiskit-addon-aqc-tensor~=0.1.2
qiskit-addon-obp~=0.1.0
scipy~=1.15.1
pyscf~=2.8.0; sys.platform != 'win32'
```
</details>

This section explores how to use `qiskit-ibm-catalog` to list programs available in Qiskit Serverless, pass inputs into these programs, run them remotely, check their status, and retrieve results and logs.
<LegacyContent>
Be sure you have authenticated to Qiskit Serverless with your [IBM Quantum account](https://quantum.ibm.com/account) (see [Deploy to IBM Quantum Platform](./serverless-first-program#deploy-to-ibm-quantum-platform) for instructions).
</LegacyContent>
<CloudContent>
Be sure you have authenticated to Qiskit Serverless using your [API key](https://quantum.cloud.ibm.com/) (see [Deploy to IBM Quantum Platform](./serverless-first-program#deploy-to-ibm-quantum-platform) for instructions).
</CloudContent>

## List programs available

You can use `QiskitServerless.list()` to fetch a list of the available programs to run with Qiskit Serverless. This includes the previously uploaded [`transpile_remote_serverless`](./serverless-first-program).

In [1]:
from qiskit_ibm_catalog import QiskitServerless

serverless = QiskitServerless()
next(
    program
    for program in serverless.list()
    if program.title == "transpile_remote_serverless"
)

QiskitFunction(transpile_remote_serverless)

## Run an uploaded program and pass inputs

First, set up your inputs. Your program has three inputs: `circuits`, `backend`, and `optimization_level`. You can use `random_circuit` to create 30 random circuits:

In [2]:
from qiskit.circuit.random import random_circuit

qc_random = [(random_circuit(4, 4, measure=True, seed=i)) for i in range(10)]
qc_random[0].draw(output="mpl", idle_wires=False)

<Image src="/images/extracted-notebook-images/serverless-run-first-workload/46108a95-11f8-4ecf-83bb-131c3e7ea4c2-0.svg" alt="Output from the previous code" />

Next, use `QiskitRuntimeService` and `least_busy` to select a `backend`:

<LegacyContent>
```python
from qiskit_ibm_runtime import QiskitRuntimeService

# If you have not previously saved your credentials, follow instructions at
# https://docs.quantum.ibm.com/guides/setup-channel#iqp
# to authenticate with your API token.
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)
```
</LegacyContent>
<CloudContent>
```python
from qiskit_ibm_runtime import QiskitRuntimeService

# If you have not previously saved your credentials, follow instructions at
# https://quantum.cloud.ibm.com/docs/guides/cloud-setup
# to authenticate with your API token.
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(backend.name)
```
</CloudContent>

Set your optimization level:

In [4]:
optimization_level = 3

Select your program with `serverless.load('PROGRAM_NAME')`:

In [5]:
transpile_remote_serverless = serverless.load("transpile_remote_serverless")

Next, pass your inputs and run it in a pythonic fashion as follows:

In [6]:
job = transpile_remote_serverless.run(
    circuits=qc_random,
    backend=backend.name,
    optimization_level=optimization_level,
)

In [7]:
job.job_id

'79487d54-5b35-464f-ad47-6a3308598412'

## Check job status

With your Qiskit Serverless `job_id`, you can check the status of running jobs. This includes the following statuses:

- **`QUEUED`**: The remote program is in the Qiskit Serverless queue. The queue priority is currently based on how much you've used Qiskit Serverless
- **`INITIALIZING`**: The remote program is starting; this includes setting up the remote environment and installing dependencies
- **`RUNNING`**: The program is running. At this stage, if you have `print()` outputs in your program, you can retrieve logs using `job.logs()`
- **`DONE`**: The program is complete, and you can retrieve data stored in `save_result()` with `job.results()`

In [8]:
job.status()

'QUEUED'

In [9]:
# This cell is hidden from users, it checks the job status
assert _ in ["QUEUED", "INITIALIZING", "RUNNING", "DONE"]  # noqa: F821

<Admonition type="tip">
Currently, the IBM Quantum workloads table only reflects Qiskit Runtime workloads. Use `job.status()` to see your Qiskit Serverless workload's current status.
</Admonition>

You've successfully run your first Qiskit Serverless program!

## Retrieve logs and results

As mentioned before, once a program is `RUNNING`, you can use `job.logs()` to fetch logs created from `print()` outputs:

In [10]:
logs = job.logs()
print(logs)

No logs yet.


At any time, you can also cancel a job:

In [11]:
job.stop()

'Job has been stopped.'

Once a program is `DONE`, you can use `job.results()` to fetch the result stored in `save_result()`:

In [12]:
# We can't get results from a cancelled job, so we'll fetch a completed one instead
completed_job = next(
    job for job in serverless.jobs() if job.status() == "DONE"
)
completed_job.result()

{'transpiled_circuits': [<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f8ccbd657d0>,
  <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7f8ccb95fe90>]}

## List previously run jobs run with Qiskit Serverless

You can use `jobs()` to list all jobs submitted to Qiskit Serverless:

In [13]:
old_jobs = serverless.jobs()
old_jobs

[<Job | 79487d54-5b35-464f-ad47-6a3308598412>,
 <Job | eadf0676-a7fa-462b-baa5-3668b61f99b4>,
 <Job | c34595c3-5d0b-49be-85da-1f6bfd707490>,
 <Job | e3bbbc67-aa72-4381-bb82-c7eebb31f099>,
 <Job | 1333d3d6-49e7-4ddd-a8f2-f61f93b6d03c>,
 <Job | 29017ed5-d682-4d6b-a7ea-43574bf998fa>,
 <Job | 3a3cb0a2-98ae-410c-807b-ff6f83904556>,
 <Job | 5b523d6d-99dd-4367-99f5-ef0063d7a14e>,
 <Job | 5e9fea0a-13f7-4755-b7ea-c4af9e7c90e2>,
 <Job | ff15878b-4b1a-48ac-9cab-d7b825f108a2>]

## Next steps

<Admonition type="info" title="Recommendations">

- Explore [compute and data management tools](./serverless-manage-resources) available to your program, including parallelization.

</Admonition>